# Загрузка и обработка файлов

## Скачивание данных, распаковка и запись

In [ ]:
'''
import os
from urllib.request import urlretrieve
import zipfile

# Создание папки data, если не существует
os.makedirs('./data', exist_ok=True)

# Пути к файлам
train_zip_path = './data/train.zip'
valid_zip_path = './data/valid.zip'

# Скачивание архивов
urlretrieve('https://www.dropbox.com/scl/fi/bel6gt6vsb3onahlxvyjc/train_fix.zip?rlkey=q2wscp6wv9j2hbk07y1mbcm54&dl=1', train_zip_path)
urlretrieve('https://www.dropbox.com/scl/fi/cwwblwhvqgwubb8a4xg90/valid.zip?rlkey=mow899lvyawq4wku2m8lfvrh3&dl=1', valid_zip_path)

# Распаковка
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall('./data/train')

with zipfile.ZipFile(valid_zip_path, 'r') as zip_ref:
    zip_ref.extractall('./data/valid')

print("Архивы скачаны и распакованы в папку ./data")
'''


'\nimport os\nfrom urllib.request import urlretrieve\nimport zipfile\n\n# Создание папки data, если не существует\nos.makedirs(\'./data\', exist_ok=True)\n\n# Пути к файлам\ntrain_zip_path = \'./data/train.zip\'\nvalid_zip_path = \'./data/valid.zip\'\n\n# Скачивание архивов\nurlretrieve(\'https://www.dropbox.com/scl/fi/bel6gt6vsb3onahlxvyjc/train_fix.zip?rlkey=q2wscp6wv9j2hbk07y1mbcm54&dl=1\', train_zip_path)\nurlretrieve(\'https://www.dropbox.com/scl/fi/cwwblwhvqgwubb8a4xg90/valid.zip?rlkey=mow899lvyawq4wku2m8lfvrh3&dl=1\', valid_zip_path)\n\n# Распаковка\nwith zipfile.ZipFile(train_zip_path, \'r\') as zip_ref:\n    zip_ref.extractall(\'./data/train\')\n\nwith zipfile.ZipFile(valid_zip_path, \'r\') as zip_ref:\n    zip_ref.extractall(\'./data/valid\')\n\nprint("Готово! Архивы скачаны и распакованы в папку ./data")\n'

## Разделим виды бабочек по классам в разные папки

In [ ]:
'''
import os
import shutil

def restructure_dataset(source_dir):
    for file_name in os.listdir(source_dir):
        if not file_name.endswith(('.jpg', '.jpeg', '.png')):
            continue

        # Извлекаем имя класса из имени файла (до первой скобки)
        class_name = file_name.split(' (')[0]
        class_dir = os.path.join(source_dir, class_name)

        # Создаем папку, если нужно
        os.makedirs(class_dir, exist_ok=True)

        # Перемещаем файл в папку
        src_path = os.path.join(source_dir, file_name)
        dst_path = os.path.join(class_dir, file_name)
        shutil.move(src_path, dst_path)

# Применяем к train и valid
restructure_dataset('./data/train')
restructure_dataset('./data/valid')

print("Структура папок исправлена")
'''

'\nimport os\nimport shutil\nfrom pathlib import Path\n\ndef restructure_dataset(source_dir):\n    for file_name in os.listdir(source_dir):\n        if not file_name.endswith((\'.jpg\', \'.jpeg\', \'.png\')):\n            continue\n\n        # Извлекаем имя класса из имени файла (до первой скобки)\n        class_name = file_name.split(\' (\')[0]\n        class_dir = os.path.join(source_dir, class_name)\n\n        # Создаем папку, если нужно\n        os.makedirs(class_dir, exist_ok=True)\n\n        # Перемещаем файл в папку\n        src_path = os.path.join(source_dir, file_name)\n        dst_path = os.path.join(class_dir, file_name)\n        shutil.move(src_path, dst_path)\n\n# Применяем к train и valid\nrestructure_dataset(\'./data/train\')\nrestructure_dataset(\'./data/valid\')\n\nprint("✅ Структура папок исправлена")\n'

# Подготовка данных

In [3]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

### Подключим необходимое число потоков (80% от возможных)

In [4]:
import os

max_threads = int(os.cpu_count() * 0.8)
os.environ["OMP_NUM_THREADS"] = str(max_threads)
os.environ["MKL_NUM_THREADS"] = str(max_threads)
torch.set_num_threads(max_threads)

In [5]:
# Аугментации и нормализация
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

transform_valid = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [6]:
# Загрузка датасета
train_data_full = datasets.ImageFolder(root='./data/train', transform=transform_train)
valid_data = datasets.ImageFolder(root='./data/valid', transform=transform_valid)

# Разделение тренировочной на обучающую и внутреннюю валидацию
train_size = int(0.8 * len(train_data_full))
val_size = len(train_data_full) - train_size
train_data, train_val_data = random_split(train_data_full, [train_size, val_size])

In [7]:
# DataLoader'ы
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(train_val_data, batch_size=32)
valid_loader = DataLoader(valid_data, batch_size=32)

In [8]:
# Число классов
num_classes = len(train_data_full.classes)
print(f"Всего классов: {num_classes}")

Всего классов: 75


# Работа с моделью

## Создание модели

In [9]:
class DeepCNN(nn.Module):
    def __init__(self, num_classes):
        super(DeepCNN, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.classifier(x)
        return x

## Обучение модели

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepCNN(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)

best_val_acc = 0
patience = 5
counter = 0

epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.3f}, Train Accuracy: {train_acc:.2f}%")

    # Валидация
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total
    print(f"Validation Accuracy: {val_acc:.2f}%")

    # Step for LR scheduler
    scheduler.step(val_acc)

    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        counter = 0
        torch.save(model.state_dict(), './best_model.pth')
        print("Model improved and saved.")
    else:
        counter += 1
        print(f"No improvement. Patience: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered!")
            break

c:\Users\Alexey\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10, Loss: 941.220, Train Accuracy: 4.60%
🎯 Validation Accuracy: 8.19%
✅ Model improved and saved.
Epoch 2/10, Loss: 857.656, Train Accuracy: 8.24%
🎯 Validation Accuracy: 12.44%
✅ Model improved and saved.
Epoch 3/10, Loss: 806.498, Train Accuracy: 11.74%
🎯 Validation Accuracy: 14.43%
✅ Model improved and saved.
Epoch 4/10, Loss: 766.699, Train Accuracy: 14.94%
🎯 Validation Accuracy: 18.96%
✅ Model improved and saved.
Epoch 5/10, Loss: 741.114, Train Accuracy: 16.40%
🎯 Validation Accuracy: 19.87%
✅ Model improved and saved.
Epoch 6/10, Loss: 720.603, Train Accuracy: 18.34%
🎯 Validation Accuracy: 21.76%
✅ Model improved and saved.
Epoch 7/10, Loss: 699.797, Train Accuracy: 20.18%
🎯 Validation Accuracy: 21.70%
🔁 No improvement. Patience: 1/5
Epoch 8/10, Loss: 688.235, Train Accuracy: 21.81%
🎯 Validation Accuracy: 24.56%
✅ Model improved and saved.


KeyboardInterrupt: 

In [ ]:
torch.save({
    'epoch': 6,  # номер последней завершённой эпохи
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict() if 'scheduler' in locals() else None,
}, './checkpoint.pth')

In [ ]:
checkpoint_path = './checkpoint.pth'
start_epoch = 0

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if checkpoint['scheduler_state_dict'] is not None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1  # начинаем со следующей эпохи
    print(f"Модель загружена. Продолжаем обучение с эпохи {start_epoch}")
else:
    print("Checkpoint не найден. Обучение начнётся с нуля.")

## Сохранение и загрузка модели

In [ ]:
torch.save(model.state_dict(), './model/butterfly_model_better.pth')
print("Модель сохранена")

Модель сохранена


# Оценка модели

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"🎯 Accuracy на валидационном наборе: {accuracy:.2f}%")


🎯 Accuracy на валидационном наборе: 68.40%
